# Food Security Data EDA

In this notebook we explore the data on food security available from the FAOSTAT database.

In [1]:
# Imports 
from pprint import pprint
from IPython.display import HTML
import pandas as pd
import plotly.express as px
import numpy as np
import os 

In [2]:
# Loading data

data_pth = os.path.join(os.pardir, 'data', 'food_security', 'Food_Security_Data_E_All_Data_(Normalized).csv')

food_security_pdf = pd.read_csv(data_pth, encoding = "ISO-8859-1")
food_security_pdf.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note
0,2,Afghanistan,21010,Average dietary energy supply adequacy (percen...,6121,Value,20002002,2000-2002,%,87,F,NaN
1,2,Afghanistan,21010,Average dietary energy supply adequacy (percen...,6121,Value,20012003,2001-2003,%,88,F,NaN
2,2,Afghanistan,21010,Average dietary energy supply adequacy (percen...,6121,Value,20022004,2002-2004,%,91,F,NaN
3,2,Afghanistan,21010,Average dietary energy supply adequacy (percen...,6121,Value,20032005,2003-2005,%,92,F,NaN
4,2,Afghanistan,21010,Average dietary energy supply adequacy (percen...,6121,Value,20042006,2004-2006,%,92,F,NaN


We will focus on _Target 2.1: Universal access to safe and nutritious food_ in relation to economy strength/development:

It has two indicators:

- Indicator 2.1.1: Prevalence of undernourishment.
- Indicator 2.1.2: Prevalence of moderate or severe food insecurity in the population.

In [3]:
# Selecting needed data 
undernourishment_pdf = food_security_pdf[(food_security_pdf['Item Code'].isin([210041])) & (food_security_pdf['Area Code'].isin([5858, 5859,9010, 9011]))] 
undernourishment_pdf['Value'] = undernourishment_pdf['Value'].apply(lambda x: float(x.replace('<', '')))
food_insecurity_pdf = food_security_pdf[(food_security_pdf['Item Code'].isin([210091])) & (food_security_pdf['Area Code'].isin([5858, 5859,9010, 9011]))] 
food_insecurity_pdf['Value'] = food_insecurity_pdf['Value'].apply(lambda x: float(x.replace('<', '')))

C:\Users\ngrec\AppData\Local\Temp/ipykernel_16956/1355017613.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undernourishment_pdf['Value'] = undernourishment_pdf['Value'].apply(lambda x: float(x.replace('<', '')))
C:\Users\ngrec\AppData\Local\Temp/ipykernel_16956/1355017613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_insecurity_pdf['Value'] = food_insecurity_pdf['Value'].apply(lambda x: float(x.replace('<', '')))


## Visualizing Indicators

### Moderate and sevier food insecurity 

In [4]:
# insecurity data 
px.line(food_insecurity_pdf, x='Year', y='Value', color='Area')

### Prevalence of Undernourishment 

In [5]:
# Undernourishment data 
px.line(undernourishment_pdf, x='Year', y='Value', color='Area')

In [ ]:
# TODO:look at absolute numbers vs prevelance and see if possible to do a T-test 

## Visualizing potential factors

### Consumer Price Indicators

In [6]:
# Loading consumer price index data

consumer_data_pth = os.path.join(os.pardir, 'data', 'food_security', 'FAOSTAT_consumer_price_indicators.csv')

cpi_pdf = pd.read_csv(consumer_data_pth)[['Area Code (M49)', 'Year', 'Item', 'Months', 'Value']]
cpi_pdf.head()


,Area Code (M49),Year,Item,Months,Value
0,4,2000,"Consumer Prices, Food Indices (2015 = 100)",January,24.356332
1,4,2001,"Consumer Prices, Food Indices (2015 = 100)",January,29.944592
2,4,2002,"Consumer Prices, Food Indices (2015 = 100)",January,33.421952
3,4,2003,"Consumer Prices, Food Indices (2015 = 100)",January,39.967661
4,4,2004,"Consumer Prices, Food Indices (2015 = 100)",January,43.401939


In [7]:
cpi_pdf[['Item']].drop_duplicates()

,Item
0,"Consumer Prices, Food Indices (2015 = 100)"
261,"Consumer Prices, General Indices (2015 = 100)"
522,Food price inflation


We see both food price indices and general price indices

In [33]:
# Joining the cpi data with economic group 

cg_data_path = os.path.join(os.pardir, 'data', 'food_security', 'FAOSTAT_country_group.csv')
cg_pdf = pd.read_csv(cg_data_path)

# Filtering out all but the X income economies Country Group
cg_pdf = cg_pdf[cg_pdf['Country Group Code'].isin([5858, 5859,9010, 9011])][['Country Group', 'M49 Code']].rename(columns={'M49 Code':'Area Code (M49)', 'Country Group': 'Economic Group'})
ecpi_pdf = cpi_pdf.merge(cg_pdf).drop(columns=['Area Code (M49)']).groupby(['Economic Group', 'Year', 'Months', 'Item']).mean().reset_index()
ecpi_pdf['date'] = pd.to_datetime(ecpi_pdf['Year'].astype(str)  + ecpi_pdf['Months'], format='%Y%B')
ecpi_pdf .head()

,Economic Group,Year,Months,Item,Value,date
0,High-income economies,2000,April,"Consumer Prices, Food Indices (2015 = 100)",63.919783,2000-04-01
1,High-income economies,2000,April,"Consumer Prices, General Indices (2015 = 100)",68.103791,2000-04-01
2,High-income economies,2000,August,"Consumer Prices, Food Indices (2015 = 100)",64.219141,2000-08-01
3,High-income economies,2000,August,"Consumer Prices, General Indices (2015 = 100)",68.561712,2000-08-01
4,High-income economies,2000,December,"Consumer Prices, Food Indices (2015 = 100)",64.758543,2000-12-01


In [35]:
px.line(ecpi_pdf[ecpi_pdf['Item'] == 'Consumer Prices, Food Indices (2015 = 100)'].sort_values(by=['date']) , x='date', y='Value', color='Economic Group')

We can see that there is some outlier in the Upper-middle-income economy group.

In [60]:
cpi_pdf = pd.read_csv(consumer_data_pth).drop(columns=['Domain Code', 'Domain', 'Item Code'])
cpi_pdf[(cpi_pdf['Year'] >= 2020) & (cpi_pdf['Item'] == 'Consumer Prices, Food Indices (2015 = 100)')  & (cpi_pdf['Value'] > 50000)].head()

,Area Code (M49),Area,Year Code,Year,Item,Months Code,Months,Unit,Value,Flag,Flag Description,Note
150764,862,Venezuela (Bolivarian Republic of),2020,2020,"Consumer Prices, Food Indices (2015 = 100)",7001,January,NaN,2.177090e+09,X,International reliable sources,base year is 2015
150765,862,Venezuela (Bolivarian Republic of),2021,2021,"Consumer Prices, Food Indices (2015 = 100)",7001,January,NaN,5.509164e+10,X,International reliable sources,base year is 2015
150786,862,Venezuela (Bolivarian Republic of),2020,2020,"Consumer Prices, Food Indices (2015 = 100)",7002,February,NaN,2.552486e+09,X,International reliable sources,base year is 2015
150787,862,Venezuela (Bolivarian Republic of),2021,2021,"Consumer Prices, Food Indices (2015 = 100)",7002,February,NaN,7.358337e+10,X,International reliable sources,base year is 2015
150808,862,Venezuela (Bolivarian Republic of),2020,2020,"Consumer Prices, Food Indices (2015 = 100)",7003,March,NaN,2.886864e+09,X,International reliable sources,base year is 2015


Looking at the above we can see that it is caused by Venezuela, we will treat this as an outlier and filter it from our data

In [57]:
ecpi_pdf = cpi_pdf[~cpi_pdf.isin([862])].merge(cg_pdf).drop(columns=['Area Code (M49)']).groupby(['Economic Group', 'Year', 'Months', 'Item']).mean().reset_index()
ecpi_pdf['date'] = pd.to_datetime(ecpi_pdf['Year'].astype(str)  + ecpi_pdf['Months'], format='%Y%B')
ecpi_pdf .head()

,Economic Group,Year,Months,Item,Year Code,Months Code,Value,date
0,High-income economies,2000,April,"Consumer Prices, Food Indices (2015 = 100)",2000.0,7004.0,63.919783,2000-04-01
1,High-income economies,2000,April,"Consumer Prices, General Indices (2015 = 100)",2000.0,7004.0,68.103791,2000-04-01
2,High-income economies,2000,August,"Consumer Prices, Food Indices (2015 = 100)",2000.0,7008.0,64.219141,2000-08-01
3,High-income economies,2000,August,"Consumer Prices, General Indices (2015 = 100)",2000.0,7008.0,68.561712,2000-08-01
4,High-income economies,2000,December,"Consumer Prices, Food Indices (2015 = 100)",2000.0,7012.0,64.758543,2000-12-01


In [58]:
px.line(ecpi_pdf[ecpi_pdf['Item'] == 'Consumer Prices, Food Indices (2015 = 100)'].sort_values(by=['date']) , x='date', y='Value', color='Economic Group')

In [62]:
px.line(ecpi_pdf[ecpi_pdf['Item'] == 'Food price inflation'].sort_values(by=['date']) , x='date', y='Value', color='Economic Group')

In [61]:
px.line(ecpi_pdf[ecpi_pdf['Item'] == 'Consumer Prices, General Indices (2015 = 100)'].sort_values(by=['date']) , x='date', y='Value', color='Economic Group')

### Income 

In [71]:
wdi_data_pth = os.path.join(os.pardir, 'data', 'food_security', 'WORLD_BANK_WDI.csv')
wdi_pdf = pd.read_csv(wdi_data_pth)
wdi_pdf.head()

,Time,Time Code,Country Name,Country Code,Adjusted net national income (current US$) [NY.ADJ.NNTY.CD],Income share held by highest 20% [SI.DST.05TH.20],Income share held by lowest 20% [SI.DST.FRST.20],Food exports (% of merchandise exports) [TX.VAL.FOOD.ZS.UN],Food imports (% of merchandise imports) [TM.VAL.FOOD.ZS.UN],Food production index (2014-2016 = 100) [AG.PRD.FOOD.XD]
0,2001,YR2001,Sweden,SWE,2.033567e+11,NaN,NaN,3.010898,7.047689,98.68
1,2001,YR2001,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,62.28
2,2001,YR2001,Albania,ALB,3.649171e+09,NaN,NaN,5.789857,19.380031,65.89
3,2001,YR2001,Algeria,DZA,4.140487e+10,NaN,NaN,0.148606,26.136152,45.04
4,2001,YR2001,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,75.09


In [73]:
wdi_pdf.isna().sum()

Time                                                              0
Time Code                                                         0
Country Name                                                      0
Country Code                                                      0
Adjusted net national income (current US$) [NY.ADJ.NNTY.CD]    1037
Income share held by highest 20% [SI.DST.05TH.20]              3029
Income share held by lowest 20% [SI.DST.FRST.20]               3029
Food exports (% of merchandise exports) [TX.VAL.FOOD.ZS.UN]    1054
Food imports (% of merchandise imports) [TM.VAL.FOOD.ZS.UN]    1040
Food production index (2014-2016 = 100) [AG.PRD.FOOD.XD]        709
dtype: int64